<a href="https://colab.research.google.com/github/beltrewilton/cx_analyzer/blob/main/Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Grupo 4  - Proyecto `luzia` 
## Descubrir Tópicos basado en la técnica Latent Dirichlet Allocation [paper](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)
### Está aplicado utilizando la base de imdb previamente limpia para ir directo al punto.

In [ ]:
!git clone https://github.com/beltrewilton/cx_analyzer.git

Cloning into 'cx_analyzer'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 26 (delta 5), reused 16 (delta 2), pack-reused 0
Unpacking objects: 100% (26/26), 22.52 MiB | 7.61 MiB/s, done.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving spanish.txt to spanish.txt


In [ ]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [ ]:
stop_spanish = get_stopwords_list("spanish.txt")
stop_spanish

stop_spanish.extend(['si','ahi','ahí', 'ah'])

In [ ]:
## read the data, separated ith ";"

data = pd.read_csv("MIT_GLS_World.csv", sep = ";",
                   encoding_errors="ignore", encoding = 'utf_8')
print(data)

     industria        nps                                            razones
0        banca    NEUTRAL           TIENEN BUENA ATENCION A LA HORA DE PAGAR
1        banca   PROMOTOR  ES RAPIDO Y SEGURO SE PUEDE REALIZAR EL PAGO D...
2        banca    NEUTRAL  A VECES EL APP DEL BIM NO FUNCIONA PARA REALIZ...
3        banca    NEUTRAL             ES MAS FACIL Y RAPIDO REALIZAR EL PAGO
4        banca   PROMOTOR          ES MAS RAPIDO Y PRACTICO DESDE EL CELULAR
...        ...        ...                                                ...
2466     banca  DETRACTOR  LA SEORITA ULTIMA QUE YO TENGO COMO A ANALISTA...
2467     banca  DETRACTOR  PORQUE ES MUCHO INTERS Y A COMO YO EMPEC A TRA...
2468     banca  DETRACTOR  PORQUE LA VERDAD YO NO RECOMENDARA, BUENO PORQ...
2469     banca  DETRACTOR  ESTOY EN UN GRUPO DE COMPARTAMOS, ESTOY EN MI ...
2470     banca  DETRACTOR  PORQUE PERTENEZCO A UN GRUPO DE COMPARTAMOS, E...

[2471 rows x 3 columns]


In [ ]:
count = CountVectorizer(stop_words= stop_spanish, max_df=0.1, max_features=5000)

In [ ]:
X = count.fit_transform(data['razones'].values)

Más información de la implementacion Dirichlet en :

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

In [ ]:
# Esta tarea NLP para descubrir los tópicos consume tiempo, paciencia y café!
n_comp = np.where(len(data. index) < 500, 8,
                  np.where(len(data. index) < 1000, 10,
                           np.where(len(data. index) < 2000, 12,
                                    np.where(len(data. index) < 3000, 15, 20))))
lda = LatentDirichletAllocation(n_components=n_comp, random_state=123, learning_method='batch')
X_topics = lda.fit_transform(X)

A continuación se hace selección de los top 5 tópicos por 10 componentes encontrados.

In [ ]:
n_top_words = 10
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_, 1):
    print(f'Topic {topic_idx}')
    print(' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

Topic 1
crdito puedo sistema celular soles hacer aunque grupo poder personas
Topic 2
inters intereses ms soles alto menos tasa ejemplo cobran prstamo
Topic 3
ahorras sencillo manera rapida momento dan poder das ms as
Topic 4
bien accesible da mejor cada sistema falla demora cobrar ms
Topic 5
agentes retirar comision cobran agente retiro monto paga partes podia
Topic 6
problemas ok practico demas pagos puede poder sistema agentes desembolso
Topic 7
bien ms compartamos hacer grupo pagos siempre mes cosas asesor
Topic 8
intereses dinero altos ms efectivo robos soles mil bien grupo
Topic 9
grupo compartamos prstamo asesor dan das personas voy as da
Topic 10
colas ahorro evita kasnet hacer agente hace buscar retiros monto
Topic 11
problema crdito ningun analista estn da inconveniente dijeron compartamos hacer
Topic 12
evito ir agencia cola ahorra hacer casa puedo ahorro practico
Topic 13
buena atencion saldo credito disponible soles kasnet compartamos pasaje ms
Topic 14
poder cualquier grup

# **First Stop!!!** Customer must review the topics and assign a name for the identified patterns!

Que sigue?
Mi recomendación es que miremos una implementación aún más robusta de Dirichlet sobre Deep Learning desarrollada por la empresa Uber llamada Pyro, más información:

*   https://medium.com/@anmolmittal_28682/evaluate-probabilistic-topic-models-pyro-latent-dirichlet-allocation-b84b73d110a7
*   https://pyro.ai/examples/intro_long.html




In [ ]:
topics = pd.DataFrame(lda.transform(X))
topics_encode = pd.DataFrame(np.where(topics >= 0.3, 1, 0))
other = pd.DataFrame(np.where(topics_encode.sum(axis = 1) == 0, 1, 0)) 
other.columns = ["other"]

In [ ]:
names = []
for topic_idx, topic in enumerate(lda.components_, 1):
    names.append(f'Topic {topic_idx}')

topics_encode.columns = names

In [ ]:
full_data = data.join(topics_encode) 
full_data = full_data.join(other)

full_data

,industria,nps,razones,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,other
0,banca,NEUTRAL,TIENEN BUENA ATENCION A LA HORA DE PAGAR,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,banca,PROMOTOR,ES RAPIDO Y SEGURO SE PUEDE REALIZAR EL PAGO D...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,banca,NEUTRAL,A VECES EL APP DEL BIM NO FUNCIONA PARA REALIZ...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
3,banca,NEUTRAL,ES MAS FACIL Y RAPIDO REALIZAR EL PAGO,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,banca,PROMOTOR,ES MAS RAPIDO Y PRACTICO DESDE EL CELULAR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,banca,DETRACTOR,LA SEORITA ULTIMA QUE YO TENGO COMO A ANALISTA...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2467,banca,DETRACTOR,PORQUE ES MUCHO INTERS Y A COMO YO EMPEC A TRA...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2468,banca,DETRACTOR,"PORQUE LA VERDAD YO NO RECOMENDARA, BUENO PORQ...",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2469,banca,DETRACTOR,"ESTOY EN UN GRUPO DE COMPARTAMOS, ESTOY EN MI ...",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
